In [2]:
import os
import random as rnd
import uuid
from concurrent.futures import ThreadPoolExecutor, as_completed

import cv2
import numpy as np
from tqdm import tqdm

In [ ]:
# Image Blurring Functions


def gaussian_blur(image, kernel_size_range=(3, 15), sigma_range=(0, 5)):
    """
    Apply Gaussian blur to an image with a random kernel size and sigma.

    Parameters:
    - image: Input image (numpy array).
    - kernel_size_range: Tuple (min, max) for the kernel size.
    - sigma_range: Tuple (min, max) for the sigma value.

    Returns:
    - Blurred image (numpy array).
    """
    # Generate random kernel size and sigma
    kernel_size = np.random.randint(kernel_size_range[0], kernel_size_range[1] + 1)
    if kernel_size % 2 == 0:
        kernel_size += 1  # Ensure kernel size is odd
    sigma = np.random.uniform(sigma_range[0], sigma_range[1])

    # Apply Gaussian blur
    blurred_image = cv2.GaussianBlur(image, (kernel_size, kernel_size), sigma)

    return blurred_image


def motion_blur(image, kernel_size_range=(5, 25)):
    """
    Apply motion blur to an image with a random kernel size.

    Parameters:
    - image: Input image (numpy array).
    - kernel_size_range: Tuple (min, max) for the kernel size.

    Returns:
    - Blurred image (numpy array).
    """
    # Generate random kernel size
    kernel_size = np.random.randint(kernel_size_range[0], kernel_size_range[1] + 1)
    if kernel_size % 2 == 0:
        kernel_size += 1  # Ensure kernel size is odd
    # Create motion blur kernel
    kernel = np.zeros((kernel_size, kernel_size))
    kernel[int((kernel_size - 1) / 2), :] = np.ones(kernel_size)
    kernel /= kernel_size
    # Apply motion blur
    blurred_image = cv2.filter2D(image, -1, kernel)

    return blurred_image


def defocus_blur(image, kernel_size_range=(5, 15)):
    """
    Apply defocus blur to an image with a random kernel size.

    Parameters:
    - image: Input image (numpy array).
    - kernel_size_range: Tuple (min, max) for the kernel size.

    Returns:
    - Blurred image (numpy array).
    """
    # Generate random kernel size
    kernel_size = np.random.randint(kernel_size_range[0], kernel_size_range[1] + 1)
    if kernel_size % 2 == 0:
        kernel_size += 1  # Ensure kernel size is odd

    # Create defocus blur kernel
    kernel = np.zeros((kernel_size, kernel_size))
    cv2.circle(kernel, (kernel_size // 2, kernel_size // 2), kernel_size // 2, 1, -1)
    kernel /= np.sum(kernel)

    # Apply defocus blur
    blurred_image = cv2.filter2D(image, -1, kernel)

    return blurred_image


def zoom_blur(image, focus=0.1, distance=4, steps=100):
    """
    Paint.NET–style zoom blur.

    Parameters:
    - image:        Input image (H×W×C, uint8).
    - distance:     Maximum zoom amount as a percentage (e.g. 20 → final scale = 1.20).
    - focus:        Exponent controlling distribution of zoom steps:
                      >1 concentrates blur near the edges;
                      <1 concentrates it near the center.
    - steps:        Number of intermediate blends (more → smoother trails).

    Returns:
    - Blurred image (uint8).
    """
    h, w = image.shape[:2]
    center = (w / 2, h / 2)
    accum = np.zeros_like(image, dtype=np.float32)

    # Precompute factor
    dist_factor = distance / 100.0

    for i in range(steps):
        t = i / (steps - 1)  # 0.0 … 1.0
        scale = 1.0 + dist_factor * (t**focus)
        rot_matrix = cv2.getRotationMatrix2D(center, 0, scale)
        warped = cv2.warpAffine(image, rot_matrix, (w, h), flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_REFLECT)
        accum += warped.astype(np.float32)

    # Average and convert back
    blurred = accum / steps
    return np.clip(blurred, 0, 255).astype(np.uint8)

In [ ]:
# Image Noise Functions


def gaussian_noise(image, strong_noise: bool = True):
    """
    Add Gaussian noise to an image.

    Parameters:
    - image: Input image (numpy array).
    - strong_noise: bool, if True, use a more intense noise range

    Returns:
    - Noisy image (numpy array).
    """
    sigma_range = (0.4, 1) if strong_noise else (0.15, 0.4)
    sigma = np.random.uniform(sigma_range[0], sigma_range[1])
    noise = np.random.normal(0, sigma, image.shape).astype(np.uint8)
    noisy_image = cv2.add(image, noise)
    noisy_image = np.clip(noisy_image, 0, 255).astype(np.uint8)
    return noisy_image


def poisson_noise(image, strong_noise: bool = False):
    """
    Add Poisson noise to an image.

    Parameters:
    - image: Input image (numpy array).
    - strong_noise: bool, if True, use a more intense noise range

    Returns:
    - Noisy image (numpy array).
    """
    lambda_range = (10, 30) if strong_noise else (30, 50)
    lam = np.random.randint(lambda_range[0], lambda_range[1] + 1)
    noisy_image = np.random.poisson(image / 255.0 * lam) / lam * 255
    noisy_image = np.clip(noisy_image, 0, 255).astype(np.uint8)
    return noisy_image


def shot_noise(image, strong_noise: bool = False):
    """
    Adds achromatic (white) Gaussian noise to an RGB image to simulate shot noise.

    Params:
    - image: H×W×3 uint8 array
    - strong_noise: bool, if True, use a more intense noise range

    Returns:
      noisy_img: H×W×3 uint8 array
    """
    sigma_range = (10, 50) if strong_noise else (5, 10)
    sigma = rnd.randint(*sigma_range)

    h, w = image.shape[:2]
    # single-channel noise
    noise = np.random.normal(0, sigma, (h, w)).astype(np.float32)
    noise = cv2.GaussianBlur(noise, (3, 3), 0)

    # broadcast into 3 channels
    noise_rgb = np.stack([noise] * 3, axis=-1)

    # add and clip
    noisy = image.astype(np.float32) + noise_rgb
    return np.clip(noisy, 0, 255).astype(np.uint8)

In [5]:
def get_image_paths(directory):
    """
    Recursively get paths to all images in a directory and its subdirectories.

    Params:
        directory (str): The base directory to search.

    Returns:
        List of image file paths.
    """
    image_paths = []

    for root, dirs, files in os.walk(directory):
        for file in files:
            # Check if the file has a valid image extension
            if any(file.lower().endswith(ext) for ext in [".jpg", ".jpeg", ".png"]):
                image_paths.append(os.path.join(root, file))

    return image_paths

In [ ]:
BASE_DIR = "../data/raw"
OUTPUT_DIR = "../data/processed"
AUGMENTATIONS_PER_IMAGE = 1
image_paths = get_image_paths(BASE_DIR)
# Add each image multiple times
image_paths = image_paths * AUGMENTATIONS_PER_IMAGE


def process_image(image_path: str):
    # Blur only, Noise only, Blur + Noise, Heavy Noise, No degradation
    thresholds = [0.1, 0.1, 0.4, 0.2, 0.2]

    target_image = cv2.imread(image_path, cv2.IMREAD_COLOR)
    degradation_type = np.random.choice(len(thresholds), p=thresholds)

    blur_funcs = [gaussian_blur, motion_blur, defocus_blur, zoom_blur]
    noise_funcs = [gaussian_noise, poisson_noise, shot_noise]

    blur_fn = rnd.choice(blur_funcs)
    noise_fn = rnd.choice(noise_funcs)
    if degradation_type == 0:
        # Apply blur
        input_image = blur_fn(target_image)
    elif degradation_type == 1:
        # Apply noise
        input_image = noise_fn(target_image)
    elif degradation_type == 2:
        # Apply blur and noise
        input_image = blur_fn(target_image)
        input_image = noise_fn(input_image)
    elif degradation_type == 3:
        # Apply heavy noise
        input_image = noise_fn(target_image, strong_noise=True)
    elif degradation_type == 4:
        # No degradation
        input_image = target_image.copy()

    image_idx = uuid.uuid4().hex
    input_filename = os.path.join(OUTPUT_DIR, "input", f"{image_idx}.jpg")
    target_filename = os.path.join(OUTPUT_DIR, "target", f"{image_idx}.jpg")
    cv2.imwrite(input_filename, input_image)
    cv2.imwrite(target_filename, target_image)

    return image_idx


with ThreadPoolExecutor(max_workers=16) as executor:
    futures = [executor.submit(process_image, path) for path in image_paths]

    image_ids = []
    with tqdm(total=len(futures), desc="Processing images") as pbar:
        for future in as_completed(futures):
            try:
                image_idx = future.result()
                image_ids.append(image_idx)
            finally:
                pbar.update(1)

for new_idx, old_idx in tqdm(enumerate(image_ids), desc="Finalizing names"):
    old_input_filename = os.path.join(OUTPUT_DIR, "input", f"{old_idx}.jpg")
    old_target_filename = os.path.join(OUTPUT_DIR, "target", f"{old_idx}.jpg")
    new_input_filename = os.path.join(OUTPUT_DIR, "input", f"{new_idx}.jpg")
    new_target_filename = os.path.join(OUTPUT_DIR, "target", f"{new_idx}.jpg")
    os.rename(old_input_filename, new_input_filename)
    os.rename(old_target_filename, new_target_filename)

Processing images: 100%|██████████| 1238/1238 [49:35<00:00,  2.40s/it] 
Finalizing names: 1238it [00:00, 1630.28it/s]
